In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
from torch import optim
from torchvision import models
import numpy as np

In [2]:
class SiameseNet(nn.Module):
    def __init__(self, hidden_size = [256, 100]):
        super(SiameseNet, self).__init__()
        self.hidden_size = hidden_size
        
        # ResNet requires the input to be the size of [batch_size, 3, 224, 224]
        self.ResNet34 = models.resnet34(pretrained=True)
        for param in self.ResNet34.parameters():
            param.requires_grad = False
        self.ResNet34.fc = nn.Linear(512, hidden_size[0])
        self.ResNet34.fc.requires_grad = True
        
        self.FC1 = nn.Linear(2 * hidden_size[0], hidden_size[1])
        self.Output = nn.Linear(hidden_size[1], 1)
    
    def forward(self, img1, img2):
        out1 = self.ResNet34(img1).view(-1, self.hidden_size[0])
        out2 = self.ResNet34(img2).view(-1, self.hidden_size[0])
        out = torch.cat([out1, out2], dim=1)
        out = F.tanh(self.FC1(out))
        out = self.Output(out)
        return out
    
    def save(self, filename='checkpoint.pt', is_best=False, epoch = ''):
        torch.save(self.state_dict(), filename)
        if is_best:
            toch.save(self.state_dict(), 'best.pt')
    
    def load(self, filename='checkpoint.pt'):
        self.load_state_dict(torch.load(filename))

In [3]:
x1 = Variable(torch.rand(32, 3, 224, 224)).cuda()
x2 = Variable(torch.rand(32, 3, 224, 224)).cuda()
diff = Variable(torch.rand(32, 1)).cuda()

In [4]:
batch_size, num_classes = 32, 3
label_type = None

model = SiameseNet().cuda()
# y_pred = model.forward(x1, x2)
criterion = nn.MSELoss()
if label_type and label_type == 'categorical':
    criterion == nn.CrossEntropyLoss()
params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(params, lr=0.0001)

losses = []

In [5]:
def mse_loss(input, target):
    return torch.sum((input - target) ** 2) / input.data.nelement()

In [6]:
for epoch in range(200):
    # Forward pass
    diff_pred = model.forward(x1, x2)
    
    # compute and print loss
    loss = mse_loss(diff_pred, diff)
    losses.append(loss.data)
    if epoch % 100 == 0:
        print(epoch, loss.data)
        
    # zero grads
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(model.forward(x1, x2))
model.save()

0 tensor(0.4023, device='cuda:0')
100 tensor(1.00000e-06 *
       9.0790, device='cuda:0')
tensor([[ 0.1652],
        [ 0.6762],
        [ 0.0152],
        [ 0.5734],
        [ 0.2580],
        [ 0.7373],
        [ 0.1691],
        [ 0.6825],
        [ 0.0103],
        [ 0.7213],
        [ 0.0739],
        [ 0.8589],
        [ 0.0918],
        [ 0.3416],
        [ 0.0079],
        [ 0.6057],
        [ 0.2454],
        [ 0.9355],
        [ 0.9538],
        [ 0.6154],
        [ 0.1043],
        [ 0.4078],
        [ 0.7000],
        [ 0.6773],
        [ 0.5595],
        [ 0.9438],
        [ 0.7610],
        [ 0.6861],
        [ 0.6761],
        [ 0.6715],
        [ 0.1691],
        [ 0.3603]], device='cuda:0')


In [7]:
model2 = SiameseNet().cuda()
model2.load()

print(model2.forward(x1, x2))

tensor([[ 0.1652],
        [ 0.6762],
        [ 0.0152],
        [ 0.5734],
        [ 0.2580],
        [ 0.7373],
        [ 0.1691],
        [ 0.6825],
        [ 0.0103],
        [ 0.7213],
        [ 0.0739],
        [ 0.8589],
        [ 0.0918],
        [ 0.3416],
        [ 0.0079],
        [ 0.6057],
        [ 0.2454],
        [ 0.9355],
        [ 0.9538],
        [ 0.6154],
        [ 0.1043],
        [ 0.4078],
        [ 0.7000],
        [ 0.6773],
        [ 0.5595],
        [ 0.9438],
        [ 0.7610],
        [ 0.6861],
        [ 0.6761],
        [ 0.6715],
        [ 0.1691],
        [ 0.3603]], device='cuda:0')
